# What is MongoDB?

MongoDB is an open-source, cross-platform, and distributed document-based database designed for ease of application development and scaling. It is a NoSQL database developed by MongoDB Inc.

MongoDB name is derived from the word "Humongous" which means huge, enormous. MongoDB database is built to store a huge amount of data and also perform fast.

MongoDB is not a Relational Database Management System (RDBMS). It's called a "NoSQL" database. It is opposite to SQL based databases where it does not normalize data under schemas and tables where every table has a fixed structure. Instead, it stores data in the collections as JSON based documents and does not enforce schemas. It does not have tables, rows, and columns as other SQL (RDBMS) databases.

The following table lists the relation between MongoDB and RDBMS terminologies.

|MongoDB (NoSQL Database)	|RDBMS (SQL Server, Oracle, etc.)|
|-------------------------|--------------------------------|
|Database	|Database|
|Collection	|Table|
|Document	Row |Record|
|Field	|Column|

In the RDBMS database, a table can have multiple rows and columns. Similarly in MongoDB, a collection can have multiple documents which are equivalent to the rows. Each document has multiple "fields" which are equivalent to the columns. Documents in a single collection can have different fields.

The following is an example of JSON based document.

![m](https://www.tutorialsteacher.com/Content/images/mongodb/document.png)

## Advantages of MongoDB

* MongoDB stores data as JSON based document that does not enforce the schema. It allows us to store hierarchical data in a document. This makes it easy to store and retrieve data in an efficient manner.
* It is easy to scale up or down as per the requirement since it is a document based database. MongoDB also allows us to split data across multiple servers.
* MongoDB provides rich features like indexing, aggregation, file store, etc.
* MongoDB performs fast with huge data.
* MongoDB provides drivers to store and fetch data from different applications developed in different technologies such as C#, Java, Python, Node.js, etc.
* MongoDB provides tools to manage MongoDB databases.

In [1]:
! pip3 install pymongo mongoengine

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pymongo
from urllib.parse import quote_plus, quote
import datetime
import pprint as pp
import pandas as pd

In [3]:
#user = "test"
#passw = "osbdet123$"
host = "localhost"

### CONNECTIONS

In [4]:
#client = pymongo.MongoClient(
#        "mongodb+srv://{0}:{1}@{2}/?retryWrites=true&w=majority" \
#            .format(user, passw, host))
client = pymongo.MongoClient(host, 27017)
print(client.list_database_names())

['admin', 'config', 'local']


### DATA SAMPLE

In [5]:
events_lst = [
    "2017-05-01 12:38:42.747|truck_geo_event|101|13|Suresh Srinivas|12|Des Moines to Chicago|Unsafe tail distance|41.62|-93.58|1|",
    "2017-05-01 12:38:42.799|truck_speed_event|101|13|Suresh Srinivas|12|Des Moines to Chicago|80|",
    "2017-05-01 12:38:42.8|truck_geo_event|102|10|George Vetticaden|4|Saint Louis to Tulsa|Normal|38.64|-90.18|1|",
    "2017-05-01 12:38:42.801|truck_speed_event|102|10|George Vetticaden|4|Saint Louis to Tulsa|65|",
]

In [6]:
print(events_lst[1])

2017-05-01 12:38:42.799|truck_speed_event|101|13|Suresh Srinivas|12|Des Moines to Chicago|80|


### DATABASES

A database is a place where data is stored in an organized way. In MongoDB, databases are used to store collections. A single MongoDB server can have multiple databases and a single MongoDB database can have multiple collections.

In [7]:
database = "trucks"

In [8]:
def connect_to_DB(db_name):
    #client = pymongo.MongoClient(
    #    "mongodb+srv://{0}:{1}@{2}/?retryWrites=true&w=majority" \
    #        .format(user, passw, host))
    client = pymongo.MongoClient(host, 27017)
    pp.pprint("Connected to the MongoDB database!\nParameters: {0}" \
        .format(client))
    db = client[db_name]
    return db

In [9]:
db = connect_to_DB(database)

('Connected to the MongoDB database!\n'
 "Parameters: MongoClient(host=['localhost:27017'], document_class=dict, "
 'tz_aware=False, connect=True)')


#### COLLECTIONS

A collection in MongoDB is similar to a table in RDBMS. MongoDB collections do not enforce schemas. Each MongoDB collection can have multiple documents. A document is equilant to row in a table in RDBMS.



In [10]:
events = db.events

In [11]:
type(events)

pymongo.collection.Collection

#### DOCUMENTS

In the RDBMS database, a table can have multiple rows and columns. Similarly in MongoDB, a collection can have multiple documents which are equivalent to the rows. Each document has multiple "fields" which are equivalent to the columns. So in simple terms, each MongoDB document is a record and a collection is a table that can store multiple documents.


##### Event types and schema  

![geo](https://raw.githubusercontent.com/georgevetticaden/sam-trucking-data-utils/master/readme-design-artifacts/truck-geo-event.png)  

![speed](https://raw.githubusercontent.com/georgevetticaden/sam-trucking-data-utils/master/readme-design-artifacts/truck-speed-event.png) 

In [12]:
events_cleaned_lst = []

for event in events_lst:
    event_dict = {}
    event_splitted = event.split("|")
    event_datetime_str = event_splitted[0]
    event_datetime_dt = datetime.datetime.strptime(
        event_datetime_str, "%Y-%m-%d %H:%M:%S.%f")
    event_source = event_splitted[1].split("_")[1]
    truck_id = event_splitted[2]
    driver_id = event_splitted[3]
    driver_name = event_splitted[4]
    route_id = event_splitted[5]
    route_name = event_splitted[6]
    
    event_dict["event"] = {
        "datetime": event_datetime_dt,
        "source": event_source,
    }
    event_dict["truck"] = {
        "id": truck_id,
    }
    event_dict["driver"] = {
        "id": driver_id,
        "name": driver_name,
    }        
    event_dict["route"] = {
        "id": route_id,
        "name": route_name,
    }      
    
    if "geo" in event_source:
        _type = event_splitted[7]
        latitude = float(event_splitted[8])
        longitude = float(event_splitted[9])
        correlation_id = event_splitted[10]
        
        event_dict["type"] = _type
        event_dict["position"] = {
            "latitude": latitude,
            "longitude": longitude,
        }
        event_dict["corr_id"] = correlation_id
        
    elif "speed" in event_source:
        speed = int(event_splitted[7])
        event_dict["speed"] = speed     
  
    events_cleaned_lst.append(event_dict)
    #pp.pprint(event_dict)

In [13]:
event = events_cleaned_lst[0]
event_id = events.insert_one(event).inserted_id
print(event_id)

654870e701044c893ffa8f16


In [14]:
type(event_id)

bson.objectid.ObjectId

In [15]:
str_event_id = str(event_id)
#str_event_id = "6366b2c9c5b6129baf54cff1"

#### FIND ONE DOCUMENT

In MongoDB, a collection represents a table in RDBMS and a document is like a record in a table. Here you will learn how to retrieve or find a single document from a collection.

In [16]:
from bson.objectid import ObjectId

pp.pprint(events.find_one({"_id": ObjectId(str_event_id)}))

{'_id': ObjectId('654870e701044c893ffa8f16'),
 'corr_id': '1',
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 747000),
           'source': 'geo'},
 'position': {'latitude': 41.62, 'longitude': -93.58},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'truck': {'id': '101'},
 'type': 'Unsafe tail distance'}


In [17]:
pp.pprint(events.find_one({"_id": str_event_id}))

None


#### SHOW COLLECTION LIST

In [18]:
print(db.list_collection_names())

['events']


#### CREATE MULTIPLE DOCUMENTS

In [19]:
result = events.insert_many(events_cleaned_lst[1:])
for result in result.inserted_ids:
    print(result)

654870ee01044c893ffa8f17
654870ee01044c893ffa8f18
654870ee01044c893ffa8f19


In [20]:
print(db.list_collection_names())

['events']


#### COUNT DOCUMENTS IN A COLLECTION

In [21]:
events.count_documents({})

4

In [22]:
events.count_documents({
    "event.source": "geo"})

2

In [23]:
events.count_documents({
    "event.source": "geo",
    "type": "Unsafe tail distance"})

1

In [24]:
events.count_documents({
    "speed": 80})

1

In [25]:
event = events.find_one(
    {"speed": 80})
event

{'_id': ObjectId('654870ee01044c893ffa8f17'),
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 799000),
  'source': 'speed'},
 'truck': {'id': '101'},
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'speed': 80}

#### QUERIES

In MongoDB, a collection represents a table in RDBMS and a document is like a record in a table. Here you will learn how to retrieve or find one or more documents from a collection.

**collection.find({ < query > }, { < fields > })**.

![OPERATORS](https://miro.medium.com/max/640/1*S4uhMlJ173rK_qOoN25j8Q.png)  
![LOGICALS](https://miro.medium.com/max/640/1*uR9igCWQyngNY0cNnLpMJg.png)

In [26]:
for event in events.find():
    pp.pprint(event)

{'_id': ObjectId('654870e701044c893ffa8f16'),
 'corr_id': '1',
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 747000),
           'source': 'geo'},
 'position': {'latitude': 41.62, 'longitude': -93.58},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'truck': {'id': '101'},
 'type': 'Unsafe tail distance'}
{'_id': ObjectId('654870ee01044c893ffa8f17'),
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 799000),
           'source': 'speed'},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'speed': 80,
 'truck': {'id': '101'}}
{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'corr_id': '1',
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 800000),
           'source': 'geo'},
 'position': {'latitude': 38.64, 'longitude': -90.18},
 'route': {'id': '4', 'name': 'Saint L

In [27]:
for event in events.find().sort("event.datetime", -1):#.limit(2):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'speed': 65,
 'truck': {'id': '102'}}
{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'corr_id': '1',
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 800000),
           'source': 'geo'},
 'position': {'latitude': 38.64, 'longitude': -90.18},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'truck': {'id': '102'},
 'type': 'Normal'}
{'_id': ObjectId('654870ee01044c893ffa8f17'),
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 799000),
           'source': 'speed'},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'speed': 80,
 'truck': {'id': '101'}}
{'_id': ObjectId('654870e701

In [28]:
for event in events.find({"speed": {"$exists": True}}):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f17'),
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 799000),
           'source': 'speed'},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'speed': 80,
 'truck': {'id': '101'}}
{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'speed': 65,
 'truck': {'id': '102'}}


In [29]:
for event in events.find({"position.latitude": 38.64}):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'corr_id': '1',
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 800000),
           'source': 'geo'},
 'position': {'latitude': 38.64, 'longitude': -90.18},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'truck': {'id': '102'},
 'type': 'Normal'}


In [30]:
d = datetime.datetime(2022, 10, 31)
for event in events.find({"event.datetime": {"$lt": d}}).sort("speed"):
    pp.pprint(event)

{'_id': ObjectId('654870e701044c893ffa8f16'),
 'corr_id': '1',
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 747000),
           'source': 'geo'},
 'position': {'latitude': 41.62, 'longitude': -93.58},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'truck': {'id': '101'},
 'type': 'Unsafe tail distance'}
{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'corr_id': '1',
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 800000),
           'source': 'geo'},
 'position': {'latitude': 38.64, 'longitude': -90.18},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'truck': {'id': '102'},
 'type': 'Normal'}
{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'route': {'id': '4', 'name': 'Sa

In [33]:
d = datetime.datetime(2022, 10, 31)
for event in events.find(
    {"event.datetime": 
        {"$lt": d}},
    {"driver.name"}):
    pp.pprint(event)

{'_id': ObjectId('654870e701044c893ffa8f16'),
 'driver': {'name': 'Suresh Srinivas'}}
{'_id': ObjectId('654870ee01044c893ffa8f17'),
 'driver': {'name': 'Suresh Srinivas'}}
{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'driver': {'name': 'George Vetticaden'}}
{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'name': 'George Vetticaden'}}


In [35]:
d = datetime.datetime(2022, 10, 31)
for event in events.find(
    {"$and": [
        {"event.datetime": {"$lt": d}},
        {"driver.name": "Suresh Srinivas"}]},
    {"driver.name", "route.name", "speed"}):
    pp.pprint(event)

{'_id': ObjectId('654870e701044c893ffa8f16'),
 'driver': {'name': 'Suresh Srinivas'},
 'route': {'name': 'Des Moines to Chicago'}}
{'_id': ObjectId('654870ee01044c893ffa8f17'),
 'driver': {'name': 'Suresh Srinivas'},
 'route': {'name': 'Des Moines to Chicago'},
 'speed': 80}


In [41]:
for event in events.find({
     "driver.name" : {"$in": ["George Vetticaden", "Edward Collins"]}}):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'corr_id': '1',
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 800000),
           'source': 'geo'},
 'position': {'latitude': 38.64, 'longitude': -90.18},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'truck': {'id': '102'},
 'type': 'Normal'}
{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'speed': 65,
 'truck': {'id': '102'}}


In [43]:
for event in events.find({
     "driver.name" : {"$in": ["George Vetticaden", "Edward Collins"]},
     "speed" : 65}):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'speed': 65,
 'truck': {'id': '102'}}


#### DATA UPDATES

We may also want to update data that we have previously added to a collection. Again, we can update one or more records. Let's say we accidentally added the wrong email to a user. Here we can use the update_one function passing the field we want to update by searching for the key/value pair and then use the $set option (preceded by a dollar sign) specifying the key and the new value. This will overwrite the initial value with the new one.

In [47]:
for event in events.find({"speed": 65}):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'speed': 65,
 'truck': {'id': '102'}}


In [48]:
to_search = {"speed": 65}
to_change = {"$set": {
    "speed": 75,
    "modified": True}}

events.update_one(to_search, to_change)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [50]:
for event in events.find({"speed": 65}):
    pp.pprint(event)

In [51]:
for event in events.find({"speed": 65}):
    pp.pprint(event)

In [57]:
# WHERE route.name LIKE 'Saint Louis%'
# WHERE route.name LIKE '%Saint Louis%'
for event in events.find({"route.name": {"$regex": "^Saint Louis"}}):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'corr_id': '1',
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 800000),
           'source': 'geo'},
 'position': {'latitude': 38.64, 'longitude': -90.18},
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'truck': {'id': '102'},
 'type': 'Normal'}
{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'modified': True,
 'route': {'id': '4', 'name': 'Saint Louis to Tulsa'},
 'speed': 75,
 'truck': {'id': '102'}}


In [58]:
to_search = {"route.name": {"$regex": "^Saint Louis"}}
to_change = {"$set": {"route.name": "Saint Gobain"}}

num = events.update_many(to_search, to_change)
print(num.modified_count, "documents updated.")

2 documents updated.


In [60]:
for event in events.find({"route.name": {"$regex": "^Saint"}}):
    pp.pprint(event)

{'_id': ObjectId('654870ee01044c893ffa8f18'),
 'corr_id': '1',
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 800000),
           'source': 'geo'},
 'position': {'latitude': 38.64, 'longitude': -90.18},
 'route': {'id': '4', 'name': 'Saint Gobain'},
 'truck': {'id': '102'},
 'type': 'Normal'}
{'_id': ObjectId('654870ee01044c893ffa8f19'),
 'driver': {'id': '10', 'name': 'George Vetticaden'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
           'source': 'speed'},
 'modified': True,
 'route': {'id': '4', 'name': 'Saint Gobain'},
 'speed': 75,
 'truck': {'id': '102'}}


In [61]:
for event in events.find({"route.name": {"$regex": "Chicago$"}}):
    pp.pprint(event)

{'_id': ObjectId('654870e701044c893ffa8f16'),
 'corr_id': '1',
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 747000),
           'source': 'geo'},
 'position': {'latitude': 41.62, 'longitude': -93.58},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'truck': {'id': '101'},
 'type': 'Unsafe tail distance'}
{'_id': ObjectId('654870ee01044c893ffa8f17'),
 'driver': {'id': '13', 'name': 'Suresh Srinivas'},
 'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 799000),
           'source': 'speed'},
 'route': {'id': '12', 'name': 'Des Moines to Chicago'},
 'speed': 80,
 'truck': {'id': '101'}}


#### INDEXES

In [62]:
events.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

In [64]:
pp.pprint(events.find({"position.latitude": 38.64}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 1,
                     'direction': 'forward',
                     'docsExamined': 4,
                     'executionTimeMillisEstimate': 0,
                     'filter': {'position.latitude': {'$eq': 38.64}},
                     'isEOF': 1,
                     'nReturned': 1,
                     'needTime': 4,
                     'needYield': 0,
                     'restoreState': 0,
                     'saveState': 0,
                     'stage': 'COLLSCAN',
                     'works': 6},
 'executionSuccess': True,
 'executionTimeMillis': 0,
 'nReturned': 1,
 'totalDocsExamined': 4,
 'totalKeysExamined': 0}


In [65]:
events.create_index('position.latitude')

'position.latitude_1'

In [66]:
events.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'position.latitude_1': {'v': 2, 'key': [('position.latitude', 1)]}}

In [67]:
pp.pprint(events.find({"position.latitude": 38.64}).explain()['executionStats'])

{'allPlansExecution': [],
 'executionStages': {'advanced': 1,
                     'alreadyHasObj': 0,
                     'docsExamined': 1,
                     'executionTimeMillisEstimate': 0,
                     'inputStage': {'advanced': 1,
                                    'direction': 'forward',
                                    'dupsDropped': 0,
                                    'dupsTested': 0,
                                    'executionTimeMillisEstimate': 0,
                                    'indexBounds': {'position.latitude': ['[38.64, '
                                                                          '38.64]']},
                                    'indexName': 'position.latitude_1',
                                    'indexVersion': 2,
                                    'isEOF': 1,
                                    'isMultiKey': False,
                                    'isPartial': False,
                                    'isSparse': False,


In [68]:
!pip3 install mongoengine

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [89]:
import mongoengine
from mongoengine import StringField, ListField, DateTimeField, DictField, IntField, BooleanField

mongoengine.disconnect()
mongoengine.connect(host = "mongodb://{0}:27017/{1}" \
    .format(host, database))

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

In [90]:
class SpeedEvent(mongoengine.Document):
    _id = StringField()
    event = DictField()
    truck = DictField()
    driver = DictField()
    route = DictField()
    speed = IntField   
    modified = BooleanField

In [91]:
speed_events_data = events.find({"event.source": "speed"})
speed_events_lst = list(speed_events_data)
speed_events_lst

[{'_id': ObjectId('654870ee01044c893ffa8f17'),
  'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 799000),
   'source': 'speed'},
  'truck': {'id': '101'},
  'driver': {'id': '13', 'name': 'Suresh Srinivas'},
  'route': {'id': '12', 'name': 'Des Moines to Chicago'},
  'speed': 80},
 {'_id': ObjectId('654870ee01044c893ffa8f19'),
  'event': {'datetime': datetime.datetime(2017, 5, 1, 12, 38, 42, 801000),
   'source': 'speed'},
  'truck': {'id': '102'},
  'driver': {'id': '10', 'name': 'George Vetticaden'},
  'route': {'id': '4', 'name': 'Saint Gobain'},
  'speed': 75,
  'modified': True}]

In [92]:
speed_events_df = pd.DataFrame.from_dict(speed_events_lst)

In [93]:
speed_events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   _id       2 non-null      object
 1   event     2 non-null      object
 2   truck     2 non-null      object
 3   driver    2 non-null      object
 4   route     2 non-null      object
 5   speed     2 non-null      int64 
 6   modified  1 non-null      object
dtypes: int64(1), object(6)
memory usage: 240.0+ bytes


In [94]:
speed_events_df.head()

,_id,event,truck,driver,route,speed,modified
0,654870ee01044c893ffa8f17,"{'datetime': 2017-05-01 12:38:42.799000, 'sour...",{'id': '101'},"{'id': '13', 'name': 'Suresh Srinivas'}","{'id': '12', 'name': 'Des Moines to Chicago'}",80,NaN
1,654870ee01044c893ffa8f19,"{'datetime': 2017-05-01 12:38:42.801000, 'sour...",{'id': '102'},"{'id': '10', 'name': 'George Vetticaden'}","{'id': '4', 'name': 'Saint Gobain'}",75,True


In [95]:
speed_events_df["driver"]

0      {'id': '13', 'name': 'Suresh Srinivas'}
1    {'id': '10', 'name': 'George Vetticaden'}
Name: driver, dtype: object

In [98]:
speed_events_df["driver"].apply(pd.Series)["name"]

0      Suresh Srinivas
1    George Vetticaden
Name: name, dtype: object

In [99]:
speed_events_df[speed_events_df["driver"].apply(pd.Series)["name"] == 'George Vetticaden']

,_id,event,truck,driver,route,speed,modified
1,654870ee01044c893ffa8f19,"{'datetime': 2017-05-01 12:38:42.801000, 'sour...",{'id': '102'},"{'id': '10', 'name': 'George Vetticaden'}","{'id': '4', 'name': 'Saint Gobain'}",75,True
